# T1485 - Data Destruction
Adversaries may destroy data and files on specific systems or in large numbers on a network to interrupt availability to systems, services, and network resources. Data destruction is likely to render stored data irrecoverable by forensic techniques through overwriting files or data on local and remote drives.(Citation: Symantec Shamoon 2012)(Citation: FireEye Shamoon Nov 2016)(Citation: Palo Alto Shamoon Nov 2016)(Citation: Kaspersky StoneDrill 2017)(Citation: Unit 42 Shamoon3 2018)(Citation: Talos Olympic Destroyer 2018) Common operating system file deletion commands such as <code>del</code> and <code>rm</code> often only remove pointers to files without wiping the contents of the files themselves, making the files recoverable by proper forensic methodology. This behavior is distinct from [Disk Content Wipe](https://attack.mitre.org/techniques/T1561/001) and [Disk Structure Wipe](https://attack.mitre.org/techniques/T1561/002) because individual files are destroyed rather than sections of a storage disk or the disk's logical structure.

Adversaries may attempt to overwrite files and directories with randomly generated data to make it irrecoverable.(Citation: Kaspersky StoneDrill 2017)(Citation: Unit 42 Shamoon3 2018) In some cases politically oriented image files have been used to overwrite data.(Citation: FireEye Shamoon Nov 2016)(Citation: Palo Alto Shamoon Nov 2016)(Citation: Kaspersky StoneDrill 2017)

To maximize impact on the target organization in operations where network-wide availability interruption is the goal, malware designed for destroying data may have worm-like features to propagate across a network by leveraging additional techniques like [Valid Accounts](https://attack.mitre.org/techniques/T1078), [OS Credential Dumping](https://attack.mitre.org/techniques/T1003), and [SMB/Windows Admin Shares](https://attack.mitre.org/techniques/T1021/002).(Citation: Symantec Shamoon 2012)(Citation: FireEye Shamoon Nov 2016)(Citation: Palo Alto Shamoon Nov 2016)(Citation: Kaspersky StoneDrill 2017)(Citation: Talos Olympic Destroyer 2018).

In cloud environments, adversaries may leverage access to delete cloud storage, cloud storage accounts, machine images, and other infrastructure crucial to operations to damage an organization or their customers.(Citation: Data Destruction - Threat Post)(Citation: DOJ  - Cisco Insider)

## Atomic Tests

### Atomic Test #1 - Windows - Overwrite file with Sysinternals SDelete
Overwrites and deletes a file using Sysinternals SDelete. Upon successful execution, "Files deleted: 1" will be displayed in
the powershell session along with other information about the file that was deleted.

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: Secure delete tool from Sysinternals must exist on disk at specified location (#{sdelete_exe})

##### Check Prereq Commands:
```powershell
if (Test-Path $env:TEMP\Sdelete\sdelete.exe) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
Invoke-WebRequest "https://download.sysinternals.com/files/SDelete.zip" -OutFile "$env:TEMP\SDelete.zip"
Expand-Archive $env:TEMP\SDelete.zip $env:TEMP\Sdelete -Force
Remove-Item $env:TEMP\SDelete.zip -Force

```

In [ ]:
Invoke-AtomicTest T1485 -TestNumbers 1 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
if (-not (Test-Path $env:TEMP\T1485.txt)) { New-Item $env:TEMP\T1485.txt -Force }
Invoke-Expression -Command "$env:TEMP\Sdelete\sdelete.exe -accepteula $env:TEMP\T1485.txt"
```

In [ ]:
Invoke-AtomicTest T1485 -TestNumbers 1

### Atomic Test #2 - macOS/Linux - Overwrite file with DDOverwrites and deletes a file using DD.
To stop the test, break the command with CTRL/CMD+C.
**Supported Platforms:** linux, macos#### Attack Commands: Run with `bash`
```bash
dd of=/var/log/syslog if=/dev/zero count=$(ls -l /var/log/syslog | awk '{print $5}') iflag=count_bytes
```

In [ ]:
Invoke-AtomicTest T1485 -TestNumbers 2

### Atomic Test #3 - Overwrite deleted data on C driveRansomEXX malware removes all deleted files using windows built-in cipher.exe to prevent forensic recover.
This process is very slow and test execution may timeout. 
https://www.cybereason.com/blog/cybereason-vs.-ransomexx-ransomware
https://support.microsoft.com/en-us/topic/cipher-exe-security-tool-for-the-encrypting-file-system-56c85edd-85cf-ac07-f2f7-ca2d35dab7e4
**Supported Platforms:** windows#### Attack Commands: Run with `command_prompt`
```command_prompt
cipher.exe /w:C:
```

In [ ]:
Invoke-AtomicTest T1485 -TestNumbers 3

## Detection
Use process monitoring to monitor the execution and command-line parameters of binaries that could be involved in data destruction activity, such as [SDelete](https://attack.mitre.org/software/S0195). Monitor for the creation of suspicious files as well as high unusual file modification activity. In particular, look for large quantities of file modifications in user directories and under <code>C:\Windows\System32\</code>.

In cloud environments, the occurrence of anomalous high-volume deletion events, such as the <code>DeleteDBCluster</code> and <code>DeleteGlobalCluster</code> events in AWS, or a high quantity of data deletion events, such as <code>DeleteBucket</code>, within a short period of time may indicate suspicious activity.


## Shield Active Defense
### Backup and Recovery 
 Make copies of key system software, configuration, and data to enable rapid system restoration. 

 Employ disk imaging, system backup, or file synchronization tools to create copies of key data on a protected backup repository.  This is typically done to capture/restore an entire system or major subsystems.
#### Opportunity
There is an opportunity to test what an adversary might do if destroyed data is selectively replaced by the defender.
#### Use Case
A defender can ensure data is backed up on a regular basis and backups are stored offline from the system.  If an adversary is detected destroying or altering data, the defender could selectively restore data from backup to see how the adversary reacts.
#### Procedures
Backup data on public facing websites and retain the files offline. In the event of data damage or loss, restore the data from backup.
Backup data on an end-user system and store offline.  If an adversary alters or deletes data on the system, restore the data using the backup copy.
In an adversary engagement situation, if an adversary deletes or alters files on a machine they are controlling, restore the data to it original state and location to see how the adversary reacts.
